In [ ]:
from graphblas import Matrix, Vector
from graphblas.semiring import any_pair
import numpy as np
from graphblas import Matrix, Vector, binary, indexunary, replace, semiring, unary


if __name__ == "__main__":

    class MockGraph:
        def __init__(self, edges, directed=True):
            self._key_to_id = {"zero": 0, "one": 1, "two": 2, "three": 3, "four": 4}
            self._id_to_key = {v: k for k, v in self._key_to_id.items()}
            self._directed = directed
            
            n = len(self._key_to_id)
            self._adj_matrix = Matrix(bool, n, n)
            for src, dst in edges:
                src_id = self._key_to_id[src]
                dst_id = self._key_to_id[dst]
                self._adj_matrix[src_id, dst_id] = True
                if not directed:
                    self._adj_matrix[dst_id, src_id] = True
        
        def get_property(self, prop_name):
            if prop_name == "offdiag":
                return self._adj_matrix
            return None
            
        def is_directed(self):
            return self._directed

    # Create a sample graph
    edges = [
        ("zero", "one"), 
        ("zero", "two"), 
        ("one", "two"), 
        ("two", "zero"), 
        ("two", "three"), 
        ("three", "three")
    ]
    G = MockGraph(edges)
    
    # Print the graph structure
    print("Graph structure:")
    for src, dst in edges:
        print(f"{src} -> {dst}")
    print()
    
    def print_bfs_traversal(source):
        """Print BFS traversal in order nodes are discovered"""
        print(f"Following is Breadth First Traversal (starting from {source})")
        
        # Get the full BFS result
        result = _bfs(G, source=source)
        
        # Reconstruct traversal order by running BFS level by level
        traversal_order = []
        discovered = set()
        source_id = G._key_to_id[source]
        
        # Start with source node
        traversal_order.append(source_id)
        discovered.add(source_id)
        
        # Process one level at a time
        for level in range(1, 5):  # Max 5 levels
            next_level = []
            current_level_result = _bfs(G, source=source, cutoff=level)
            
            # Find nodes at this level
            for i, val in enumerate(current_level_result):
                if val and i not in discovered:
                    next_level.append(i)
                    discovered.add(i)
            
            # Add this level's nodes to traversal
            traversal_order.extend(next_level)
            
            if not next_level:
                break
        
        # Print traversal order as node IDs (to match the example output format)
        print(" ".join(str(i) for i in traversal_order))
        
        # Also print with node names for better readability
        print(" ".join(G._id_to_key[i] for i in traversal_order))
    
    # Test BFS traversal from different source nodes
    print_bfs_traversal("two")
    print()
    print_bfs_traversal("zero")
    print()
    print_bfs_traversal("one")
    print()
    
    # Test with specific cutoff
    source = "zero"
    cutoff = 1
    result = _bfs(G, source=source, cutoff=cutoff)
    print(f"BFS from {source} with cutoff={cutoff}:")
    for i, val in enumerate(result):
        if val:
            print(f"Node {G._id_to_key[i]} is reachable")